In [1]:
# Import libraries
import psycopg2
import getpass
import numpy as np
import os.path
import datetime

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
port = '5433'
dbname = 'mimic'
schema = 'mimiciii'

In [3]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, port=port, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))
con.commit()
print("Connected!")

Password: ··········


Connected!


In [4]:
# Load admission_ids
_adm = np.load('res/admission_ids.npy', allow_pickle=True).tolist()
admission_ids = _adm['admission_ids']

# Load item_ids
db = np.load('res/item_ids.npy', allow_pickle=True).tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

print("Loaded!")

Loaded!


In [5]:
# Query the database for the units of all item IDs, and how many entries use each unit
start = datetime.datetime.now()
input_itemid_units = []
for i in input_itemid:
    if i >= 200000:
        sql = \
        f'''
        SELECT amountuom, COUNT(*)
        FROM inputevents_mv
        WHERE itemid={i} AND hadm_id IN (
            SELECT *
            FROM admission_ids)
        GROUP BY amountuom
        '''
    elif i >= 30000 and i <= 49999:
        sql = \
        f'''
        SELECT amountuom, COUNT(*)
        FROM inputevents_cv
        WHERE itemid={i} AND hadm_id IN (
            SELECT *
            FROM admission_ids)
        GROUP BY amountuom
        '''
    cur.execute(sql)
    res = cur.fetchall()
    input_itemid_units.append(res)
end = datetime.datetime.now()

In [6]:
# Make sure everything is working
print(end-start)
for i in range(10):
    print(input_itemid_units[i])

0:01:22.665615
[('ml', 8)]
[(None, 5), ('ml', 18)]
[('ml', 111)]
[('ml', 40)]
[(None, 1), ('ml', 8)]
[('ml', 83)]
[(None, 1), ('ml', 1)]
[(None, 1), ('ml', 1)]
[(None, 1), ('ml', 14)]
[(None, 1), ('ml', 4)]


In [47]:
# Construct a list with each row = [item ID, no. of entries, (unit1, percentage of entries using unit1), (unit2, etc...)]
start = datetime.datetime.now()
input_itemid_units_percent = []
for row in input_itemid_units:
    # this loop calculates the total number of entries for each item ID
    total = 0
    for unit_tuple in row:
        entries = unit_tuple[1]
        total += entries
    # this loop calculates the percentage of that item ID that each unit makes up
    new_row = [input_itemid[input_itemid_units.index(row)], total]
    for unit_tuple in row:
        percentage = (unit_tuple[1]/total) * 100
        if unit_tuple[0] != None:
            new_row.append((unit_tuple[0],percentage))
    # rearrange new_row so most common units are listed first
    new_row = new_row[:2] + sorted(new_row[2:], key=lambda tup : tup[1], reverse=True)
    input_itemid_units_percent.append(new_row)
end = datetime.datetime.now()

In [48]:
# Make sure everything is working
print(end-start)
for i in range(10):
    print(input_itemid_units_percent[i])

0:00:00.099595
[44377, 8, ('ml', 100.0)]
[30367, 23, ('ml', 78.26086956521739)]
[44743, 111, ('ml', 100.0)]
[46126, 40, ('ml', 100.0)]
[45431, 9, ('ml', 88.88888888888889)]
[46236, 83, ('ml', 100.0)]
[42866, 2, ('ml', 50.0)]
[42866, 2, ('ml', 50.0)]
[41619, 15, ('ml', 93.33333333333333)]
[44127, 5, ('ml', 80.0)]


In [55]:
filtered_input = []
less_than_90 = []
no_entries = []
for row in input_itemid_units_percent:
    # cases with 2 or more units
    if len(row) > 3:
        if row[2][1] >= 90.0:
            filtered_input.append(row)
        else:
            less_than_90.append(row)
    # cases with only one unit
    elif len(row) == 3:
        filtered_input.append(row)   
    else:
        no_entries.append(row)

for row in less_than_90:
    print(row)


[30052, 6227, ('gm', 41.0791713505701), ('ml', 0.22482736470210374)]
[30163, 40996, ('mg', 40.6966533320324), ('ml', 0.07317787101180602)]
[30089, 147509, ('tsp', 82.36853344541689), ('gm', 17.63146655458311)]
[225913, 997, ('dose', 88.26479438314945), ('mg', 11.634904714142428), ('grams', 0.10030090270812438)]
[225845, 1475, ('dose', 69.42372881355932), ('mg', 30.508474576271187), ('grams', 0.06779661016949153)]
[221744, 86340, ('mcg', 79.94556404910817), ('mg', 20.054435950891822)]
[30178, 32265, ('mg', 39.63427863009453), ('ml', 2.2408182240818224)]
[30046, 158, ('ml', 53.79746835443038), ('mg', 20.253164556962027)]
[30069, 2619, ('mg', 72.58495609011074), ('ml', 26.46048109965636)]
[30174, 508, ('ml', 33.661417322834644), ('mg', 30.905511811023622)]
[30151, 2471, ('mg', 47.875354107648725), ('ml', 8.984216916228247)]
[225914, 109, ('mcg', 87.1559633027523), ('dose', 7.339449541284404), ('mg', 3.669724770642202), ('pg', 1.834862385321101)]
[30172, 51532, ('mg', 43.805790576729024), 

In [ ]:
# Save the item IDs with corresponding units and percentages
np.save('res/input_raw.npy', {'raw' : input_itemid_units_percent})
print('Saved!')